# Imports

In [1]:
import os
import math
from pathlib import Path

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter

import numpy as np

# Constants

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
BATCH_SIZE = 64
NUM_WORKERS = 0

In [4]:
ROOT_DIR = Path('.')
DATA_DIR = ROOT_DIR / 'data'
REPORTS_DIR = ROOT_DIR / 'reports'
MODELS_DIR = REPORTS_DIR / 'models'
RESULTS_DIR = REPORTS_DIR / 'results'
RUNS_DIR = REPORTS_DIR / 'runs'

# Data loading

In [5]:
train_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5)]
)
val_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5)]
)
test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]
)

In [6]:
train_proportion = 0.9
num_train = 50000

indices = list(range(num_train))
split = int(np.floor(train_proportion * num_train))
np.random.shuffle(indices)

train_idx, val_idx = indices[:split], indices[split:]
train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)

In [7]:

train_dataset = datasets.CIFAR10(root=DATA_DIR, train=True,
                                 download=True, transform=train_transform)

val_dataset = datasets.CIFAR10(root=DATA_DIR, train=True,
                               download=True, transform=val_transform)

test_dataset = datasets.CIFAR10(root=DATA_DIR, train=False,
                                download=True, transform=test_transform)

100%|██████████| 170M/170M [00:02<00:00, 84.9MB/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified
Files already downloaded and verified


In [8]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                          sampler=train_sampler, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                        sampler=val_sampler, num_workers=NUM_WORKERS)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False,
                         num_workers=NUM_WORKERS)

# Aggregating functions

In [9]:
def arithmetic_mean(X, dim, keepdim):
    return torch.mean(X, dim, keepdim)

In [10]:
def minimum(X, dim, keepdim):
    return torch.min(X, dim, keepdim).values

In [11]:
def product(X, dim, keepdim):
    return torch.prod(X, dim=dim, keepdim=keepdim)

In [12]:
def t_norm_lukasiewicz(X, dim, keepdim):
    sum_X = torch.sum(X, dim=dim, keepdim=keepdim) - 1
    return torch.max(sum_X, torch.tensor(0, device=X.device))

In [13]:
def t_norm_hamacker(X, dim, keepdim=False):
    # Check if tensor has at least 2 elements along the specified dimension
    if X.shape[dim] < 2:
        raise ValueError("Tensor must have at least 2 elements along the specified dimension.")

    # Split the tensor into pairs of values along the specified dimension
    tensor1 = X.narrow(dim, 0, 1)  # First element along the dimension
    tensor2 = X.narrow(dim, 1, 1)  # Second element along the dimension

    # Compute the Hamacher T-norm based on the formula
    zero_mask = (tensor1 == 0) & (tensor2 == 0)
    t_norm_result = torch.where(
        zero_mask,
        torch.zeros_like(tensor1),
        (tensor1 * tensor2) / (tensor1 + tensor2 - tensor1 * tensor2)
    )

    # Reduce along the specified dimension
    result = torch.prod(t_norm_result, dim=dim, keepdim=keepdim)

    return result

In [14]:
def maximum(X, dim, keepdim):
    return torch.max(X, dim, keepdim).values

In [15]:
def t_conorm_lukasiewicz(X, dim, keepdim):
    sum_X = torch.sum(X, dim=dim, keepdim=keepdim)
    return torch.min(sum_X, torch.tensor(1.0, device=X.device))

In [16]:
def t_conorm_hamacker(X, dim, keepdim):
    product_X = torch.prod(X, dim=dim, keepdim=keepdim)

    sum_X = torch.sum(X, dim=dim, keepdim=keepdim)

    numerator = 2 * product_X - sum_X
    denominator = product_X - 1

    result = torch.where(
        product_X == 1,
        torch.tensor(1.0),
        numerator / (denominator + 1e-10)  # Adding a small value for numerical stability.
    )

    return result

In [17]:
def u_min_max(X, dim, keepdim):
    # Ensure we have at least two elements along the specified dimension
    if X.shape[dim] < 2:
        raise ValueError("Tensor must have at least 2 elements along the specified dimension.")

    # Shift the tensor to create pairs of adjacent elements along the specified dimension
    tensor1 = X.narrow(dim, 0, X.shape[dim] - 1)  # Exclude the last element
    tensor2 = X.narrow(dim, 1, X.shape[dim] - 1)  # Exclude the first element

    # Condition to check if both values are in the range [0, 0.5]
    condition = (tensor1 >= 0) & (tensor1 <= 0.5) & (tensor2 >= 0) & (tensor2 <= 0.5)

    # Apply min where the condition is true, otherwise apply max
    result = torch.where(condition, torch.min(tensor1, tensor2), torch.max(tensor1, tensor2))

    # If keepdims is True, expand the reduced dimension to size 1
    if keepdim:
        result = result.unsqueeze(dim)

    return result

In [18]:
# ChatGPT version
def u_min_max_chatgpt(X, dim, keepdim):
   is_in_lower_range = (X <= 0.5).all(dim=dim, keepdim=keepdim)

   min_X = torch.min(X, dim=dim, keepdim=keepdim).values
   max_X = torch.max(X, dim=dim, keepdim=keepdim).values

   return torch.where(is_in_lower_range, min_X, max_X)

In [19]:
def u_l_l(X, dim, keepdim):
    # Ensure we have at least two elements along the specified dimension
    if X.shape[dim] < 2:
        raise ValueError("Tensor must have at least 2 elements along the specified dimension.")

    # Shift the tensor to create pairs of adjacent elements along the specified dimension
    tensor1 = X.narrow(dim, 0, X.shape[dim] - 1)  # Exclude the last element
    tensor2 = X.narrow(dim, 1, X.shape[dim] - 1)  # Exclude the first element

    # Conditions
    condition1 = (tensor1 >= 0) & (tensor1 <= 0.5) & (tensor2 >= 0) & (tensor2 <= 0.5)
    condition2 = (tensor1 >= 0.5) & (tensor1 <= 1) & (tensor2 >= 0.5) & (tensor2 <= 1)

    # Calculate each case
    result1 = torch.max(tensor1 + tensor2 - 1, torch.zeros_like(tensor1))  # max(x + y - 1, 0)
    result2 = torch.min(tensor1 + tensor2, torch.ones_like(tensor1))       # min(x + y, 1)
    result3 = torch.max(tensor1, tensor2)                                  # max(x, y)

    # Apply conditions
    result = torch.where(condition1, result1, torch.where(condition2, result2, result3))

    # Handle keepdims option
    if keepdim:
        result = result.unsqueeze(dim)

    return result

In [20]:
class ChoquetLayer(nn.Module):
    def __init__(self, input_size):
        super(ChoquetLayer, self).__init__()
        self.input_size = input_size
        self.w = None

    def forward(self, x, dim, keepdim):
        input_size = x.size(-1)
        batch_size = x.size(0)

        if self.w is None or self.w.size(0) != batch_size:
            self.w = nn.Parameter(torch.randn(input_size))

        x_sorted, indices = torch.sort(x, descending=True, dim=-1)  # (batch_size, input_size)
        v_Ai = self.compute_v_Ai(indices)  # (batch_size, n+1)
        v_delta = v_Ai[..., :-1] - v_Ai[..., 1:]  # (batch_size, input_size)
        x = torch.sum(x_sorted * v_delta, dim=-1)  # (batch_size)

        if keepdim:
          x.unsqueeze(dim)

        return x

    def compute_v_Ai(self, indices):
        w_sorted = self.w[indices]
        v_Ai = torch.cumsum(w_sorted, dim=-1)
        zeros = torch.zeros(*v_Ai.shape[:-1], 1).to(v_Ai.device)
        v_Ai = torch.cat((zeros, v_Ai), dim=-1)
        return v_Ai

# Layers

In [21]:
class AggPoolingLayer(nn.Module):

    def __init__(self, function, kernel_size, stride, padding= [0,0,0,0], dim = -1, keepdim = False):
        super().__init__()

        # Una tupla de 2 elementos con los tamaños [𝑘1,𝑘2] de cada ventana a tratar
        self.kernel_size = kernel_size

        # Tupla de 2 elementos que indican el número de elementos (en filas y columnas) que
        # deben saltarse tras reducir cada ventana, hasta encontrar la siguiente a tratar.
        self.stride = stride

        # Tupla de 4 elementos de la forma [𝑝𝑎𝑑_𝑙𝑒𝑓𝑡,𝑝𝑎𝑑_𝑟𝑖𝑔ℎ𝑡,𝑝𝑎𝑑_𝑢𝑝,𝑝𝑎𝑑_𝑑𝑜𝑤𝑛] que indica el
        # número de nuevas filas o columnas a añadir a la entrada, previo a aplicar la agregación.
        self.padding = padding

        # Define function and characteristics
        self.function = function
        self.dim = dim
        self.keepdim = keepdim

    def forward(self, X):

        # Normalize
        maximum = torch.max(X)
        minimum = torch.min(X)
        X = (X-minimum)/(maximum-minimum)

        # Añadir columnas/filas según padding
        X_pad = F.pad(X, pad=self.padding, mode='constant', value=0)

        # Vamos extrayendo las ventanas a agregar y colocándolas en filas
        X_aux = X_pad.unfold(2, size=self.kernel_size[0], step=self.stride[0]).unfold(3, size=self.kernel_size[1], step=self.stride[1])

        # Ponemos el formato correcto
        X_aux = X_aux.reshape([X_aux.shape[0], X_aux.shape[1], X_aux.shape[2], X_aux.shape[3], X_aux.shape[4] * X_aux.shape[5]])

        # Agg Func
        Y_temp = self.function(X_aux, dim = self.dim, keepdim = self.keepdim)

        # Denormalize
        Y = minimum + (maximum-minimum) * Y_temp

        return Y

In [22]:
class OWAPoolingLayer(nn.Module):

    def __init__(self, kernel_size, stride, padding= [0,0,0,0], dim = -1, keepdim = False):
        super().__init__()

        # Una tupla de 2 elementos con los tamaños [𝑘1,𝑘2] de cada ventana a tratar
        self.kernel_size = kernel_size

        # Tupla de 2 elementos que indican el número de elementos (en filas y columnas) que
        # deben saltarse tras reducir cada ventana, hasta encontrar la siguiente a tratar.
        self.stride = stride

        # Tupla de 4 elementos de la forma [𝑝𝑎𝑑_𝑙𝑒𝑓𝑡,𝑝𝑎𝑑_𝑟𝑖𝑔ℎ𝑡,𝑝𝑎𝑑_𝑢𝑝,𝑝𝑎𝑑_𝑑𝑜𝑤𝑛] que indica el
        # número de nuevas filas o columnas a añadir a la entrada, previo a aplicar la agregación.
        self.padding = padding

        # Define characteristics
        self.dim = dim
        self.keepdim = keepdim

        # Weights
        self.weight = nn.Parameter(torch.ones(1, self.kernel_size[0] * self.kernel_size[1]))


    def funcionOWA(self, X):

        tensor_ordered = torch.sort(X, descending = True)
        weight_norm = torch.nn.functional.softmax(self.weight, dim = self.dim)
        output = torch.sum(tensor_ordered[0] * weight_norm, dim = self.dim, keepdim = self.keepdim)

        return output

    def forward(self, X):

        # Normalize
        maximum = torch.max(X)
        minimum = torch.min(X)
        X = (X-minimum)/(maximum-minimum)

        # Añadir columnas/filas según padding
        X_pad = F.pad(X, pad=self.padding, mode='constant', value=0)

        # Vamos extrayendo las ventanas a agregar y colocándolas en filas
        X_aux = X_pad.unfold(2, size=self.kernel_size[0], step=self.stride[0]).unfold(3, size=self.kernel_size[1], step=self.stride[1])

        # Ponemos el formato correcto
        X_aux = X_aux.reshape([X_aux.shape[0], X_aux.shape[1], X_aux.shape[2], X_aux.shape[3], X_aux.shape[4] * X_aux.shape[5]])

        # Agg Func
        Y_temp = self.funcionOWA(X_aux)

        # Denormalize
        Y = minimum + (maximum-minimum) * Y_temp

        return Y

# Model

In [23]:
class LeNetModel(nn.Module):

    def __init__(self, function, conv_filters=[64, 64], linear_sizes=[384, 192], num_classes=10):
        super(LeNetModel, self).__init__()
        self.conv1 = nn.Conv2d(3, conv_filters[0], [2,2], [1,1])
        self.pool1 = AggPoolingLayer(function, [2,2], [2,2])
        self.conv2 = nn.Conv2d(conv_filters[0], conv_filters[1], [2,2], [1,1])
        self.pool2 = AggPoolingLayer(function, [2,2], [2,2])
        self.fc1 = nn.Linear(conv_filters[1]*7*7, linear_sizes[0])
        self.fc2 = nn.Linear(linear_sizes[0], linear_sizes[1])
        self.fc3 = nn.Linear(linear_sizes[1], num_classes)

    def forward(self, x: torch.Tensor):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = x.flatten(start_dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Training

In [24]:
writer = SummaryWriter(log_dir=RUNS_DIR)

In [28]:
def train(model, train_loader, criterion, optimizer, val_loader=None, num_epochs=20, device=device):
    train_acc = []
    train_loss = []
    # if val_loader is not None:
    val_acc = []
    val_loss = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        count_evaluated = 0
        count_correct = 0

        for batch_idx, data in enumerate(train_loader, 0):
            model.train()
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            count_evaluated += inputs.shape[0]
            count_correct += torch.sum(labels == torch.max(outputs, dim=1)[1])

        print('Training: [%d, %5d] loss: %.3f' % (epoch + 1, batch_idx + 1, running_loss / (batch_idx+1)))

        train_loss.append(running_loss / (batch_idx+1))
        train_acc.append(float(count_correct) / count_evaluated)

        if val_loader is not None:
            running_loss_val = 0.0
            count_evaluated = 0
            count_correct = 0
            model.eval()

            with torch.no_grad():
                for val_batch_idx, data_val in enumerate(val_loader, 0):
                    inputs_val, labels_val = data_val[0].to(device), data_val[1].to(device)
                    outputs_val = model(inputs_val)
                    loss = criterion(outputs_val, labels_val)
                    running_loss_val += loss.item()
                    count_evaluated += inputs_val.shape[0]
                    count_correct += torch.sum(labels_val == torch.max(outputs_val, dim=1)[1])

                val_loss.append(running_loss_val / (val_batch_idx + 1))
                acc_val = float(count_correct) / count_evaluated

                print('Validation: epoch %d - acc: %.3f' %
                            (epoch + 1, acc_val))
                val_acc.append(acc_val)

        # Tensorboard
        writer.add_scalar('Loss/Validation', val_loss[-1], global_step=epoch)
        writer.add_scalar('Accuracy/Validation', val_acc[-1], global_step=epoch)
        writer.add_scalar('Loss/Train', train_loss[-1], global_step=epoch)
        writer.add_scalar('Accuracy/Train', train_acc[-1], global_step=epoch)
        for name, param in model.named_parameters():
            writer.add_histogram(f"Parameters/{name}", param, epoch)
            if param.grad is not None:
                writer.add_histogram(f"Gradients/{name}", param.grad, epoch)

    return model

# Testing

In [29]:
def test(model, test_loader, criterion, device='cuda'):
    with torch.no_grad():
        number_samples = 0
        number_correct = 0
        running_loss_test = 0.0
        for test_batch_idx, data_test in enumerate(test_loader, 0):
            inputs_test, labels_test = data_test[0].to(device), data_test[1].long().to(device)
            outputs_test = model(inputs_test)
            loss = criterion(outputs_test, labels_test)
            running_loss_test += loss.cpu().numpy()

            _, outputs_class = torch.max(outputs_test, dim=1)
            number_correct += torch.sum(outputs_class == labels_test).cpu().numpy()
            number_samples += len(labels_test)

        acc_test = number_correct / number_samples

        print('Test - Accuracy: %.3f' % acc_test)
        print('Test - CrossEntropy: %.3f' % (running_loss_test / (test_batch_idx+1)))

In [ ]:
model = LeNetModel(arithmetic_mean)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
trained_model = train(model, train_loader, criterion, optimizer, val_loader=val_loader)

Training: [1,   704] loss: 1.568
Validation: epoch 1 - acc: 0.511
Training: [2,   704] loss: 1.231
Validation: epoch 2 - acc: 0.585
Training: [3,   704] loss: 1.085
Validation: epoch 3 - acc: 0.618
Training: [4,   704] loss: 0.984
Validation: epoch 4 - acc: 0.654
Training: [5,   704] loss: 0.904
Validation: epoch 5 - acc: 0.662
Training: [6,   704] loss: 0.843
Validation: epoch 6 - acc: 0.675
Training: [7,   704] loss: 0.791
Validation: epoch 7 - acc: 0.686
Training: [8,   704] loss: 0.744
Validation: epoch 8 - acc: 0.698
Training: [9,   704] loss: 0.700
Validation: epoch 9 - acc: 0.707
Training: [10,   704] loss: 0.656
Validation: epoch 10 - acc: 0.699
Training: [11,   704] loss: 0.621
Validation: epoch 11 - acc: 0.722
Training: [12,   704] loss: 0.587
Validation: epoch 12 - acc: 0.712
Training: [13,   704] loss: 0.558
Validation: epoch 13 - acc: 0.718
Training: [14,   704] loss: 0.531
Validation: epoch 14 - acc: 0.724
Training: [15,   704] loss: 0.504
Validation: epoch 15 - acc: 0.73